# GraphRAG Relevance, Factuality and Synthesis Evaluation

## Step 0: environment set up

In [ ]:
from dotenv import load_dotenv
import os
from langchain_neo4j import Neo4jGraph
from libs import create_vector_index, generate_response
import pandas as pd
from conn import connect2Googlesheet, get_relevant_documents , 
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
#from rouge_score import rouge_scorer
from sklearn.metrics import precision_score, recall_score, accuracy_score
# Force reload of the .env file
load_dotenv()

True

In [2]:
# Connect to Neo4j database
try:
    graph = Neo4jGraph(
        url=os.getenv("NEO4J_URL"),
        username=os.getenv("NEO4J_USERNAME"),
        password=os.getenv("NEO4J_PASSWORD")
    )
    print("Connected to Neo4j database successfully.")
except ValueError as e:
    print(f"Could not connect to Neo4j database: {e}")

Connected to Neo4j database successfully.


## Step 1: Create vector index

In [4]:
create_vector_index(graph, "entities")

Index 'entities' exists but has incorrect dimensions: None. Recreating...


## Step 2: Load questions from google sheet

In [50]:
spreadsheet = connect2Googlesheet()

# Select the worksheet: relevance
worksheet = spreadsheet.get_worksheet(2)  

# Get all records as a list of dictionaries
data = worksheet.get_all_records()

# Convert to Pandas DataFrame
df_MedQ = pd.DataFrame(data)
df_MedQ.head()

,condition,number,docs,Question,Mahmud's Note,status,comments,
0,ARDS,1,ACURASYS,Does early administration of neuromuscular blo...,Like,,,
1,ARDS,2,ACURASYS,Do patients with severe ARDS being treated wit...,Replace,fixed,,
2,ARDS,3,ROSE,"In patients with moderate to severe ARDS, does...",Maybe this question: In patients with moderate...,fixed,,
3,ARDS,4,ROSE,Do patients with moderate-to-severe ARDS have ...,Local question (not sure if this is the aim of...,fixed,Wrong concept since PEEP by itself is mandator...,Does the use of neuromuscular blockers in pati...
4,ARDS,5,FACTT,"Among patients with ALI/ARDS, does a conservat...",Local question (not sure if this is the aim of...,fixed,Check if studies defined conservative by CVP <...,


## Step 3: Relevance check for top K questions

In [8]:
topk = 5
results_df = retrieval_rel_docs(graph, df_MedQ, top_k=topk)
results_df.to_csv('retrieved_docs_results.csv', index=False)

NameError: name 'retrieval_rel_docs' is not defined

In [10]:
# load the relevant documents
results_df = pd.read_csv('retrieved_docs_results.csv')
relevant_docs_df = pd.read_csv('relevant_docs.csv').rename(columns={'Docs': 'Relevant Docs'})
concatenated_df = get_concatenate_df(results_df, relevant_docs_df, topk)
concatenated_df.head()

Error in get_concatenate_df: name 'clean_text' is not defined


""


In [ ]:

# Function to calculate accuracy, precision, and recall
def calculate_metrics(reference, candidate):
    reference_tokens = set(reference.split())
    candidate_tokens = set(candidate.split())
    
    true_positives = len(reference_tokens & candidate_tokens)
    false_positives = len(candidate_tokens - reference_tokens)
    false_negatives = len(reference_tokens - candidate_tokens)
    
    accuracy = true_positives / (true_positives + false_positives + false_negatives)
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    
    return accuracy, precision, recall

# Calculate metrics for all questions and add to the DataFrame
metrics = concatenated_df.apply(
    lambda row: calculate_metrics(str(row['Relevant Docs']), str(row['Generated Docs'])), axis=1
)

# Extract accuracy, precision, and recall and add them as new columns
concatenated_df['Accuracy'] = metrics.apply(lambda metric: metric[0])
concatenated_df['Precision'] = metrics.apply(lambda metric: metric[1])
concatenated_df['Recall'] = metrics.apply(lambda metric: metric[2])
# Display the concatenated DataFrame with BLEU Score
concatenated_df
# Save the concatenated DataFrame to a CSV file
#concatenated_df.to_csv('concatenated_results.csv', index=False)

,Question,Question,Relevant Docs,Generated Docs,Accuracy,Precision,Recall
0,Relevace Q1,Relevace Q1,ROSE ACURASYS,ACURASYS,0.50,1.0,0.50
1,Relevace Q2,Relevace Q2,ROSE ACURASYS,ACURASYS,0.50,1.0,0.50
2,Relevace Q3,Relevace Q3,ROSE,ARDS ACURASYS,0.00,0.0,0.00
3,Relevace Q4,Relevace Q4,ROSE,ROSE ACURASYS,0.50,0.5,1.00
4,Relevace Q5,Relevace Q5,FACTT,ARDS ACURASYS,0.00,0.0,0.00
5,Relevace Q6,Relevace Q6,FACTT,FACTT,1.00,1.0,1.00
6,Relevace Q7,Relevace Q7,ARDSNET,SYNDROME RESPIRATORY DISTRESS NETWORK ACUTE THE,0.00,0.0,0.00
7,Relevace Q8,Relevace Q8,ARDSNET,ARDS,0.00,0.0,0.00
8,Relevace Q9,Relevace Q9,PROSEVA,ACURASYS,0.00,0.0,0.00
9,Relevace Q10,Relevace Q10,OSCILLATE,,0.00,0.0,0.00
